In [7]:
# Procesamiento de Datos para Competición Kaggle - Precios de Viviendas en Galicia
# =============================================================================

# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import os

# Configuración de visualización
#plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
pd.set_option('display.max_columns', None)

# Definir una semilla para reproducibilidad
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# =============================================================================
# 1. CARGA DE DATOS
# =============================================================================

# Verificar si existen los archivos procesados
if os.path.exists('train_processed.csv') and os.path.exists('test_processed.csv'):
    print("Cargando datos procesados previamente...")
    train_data = pd.read_csv('train_processed.csv')
    test_data = pd.read_csv('test_processed.csv')
    print("Datos procesados cargados correctamente.")
else:
    print("Procesando datos desde cero...")
    
    # Cargar los conjuntos de datos originales
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('test.csv')
    
    print(f"Dimensiones del conjunto de entrenamiento: {train_data.shape}")
    print(f"Dimensiones del conjunto de prueba: {test_data.shape}")

Cargando datos procesados previamente...
Datos procesados cargados correctamente.


In [8]:
# =============================================================================
# 2. FUNCIÓN AUXILIAR PARA PREPROCESAMIENTO
# =============================================================================

def preprocess_data(df, is_train=True):
    """
    Función para preprocesar los datos de viviendas
    
    Parámetros:
    df (pandas.DataFrame): DataFrame a procesar
    is_train (bool): Indica si el DataFrame es el conjunto de entrenamiento
    
    Retorna:
    pandas.DataFrame: DataFrame procesado
    """
    # Crear una copia para no modificar el original
    data = df.copy()
    
    # =============================================================================
    # 2.1 LIMPIEZA DE DATOS
    # =============================================================================
    
    print("Iniciando limpieza de datos...")
    
    # Eliminar duplicados en el conjunto de entrenamiento
    if is_train:
        duplicates = data.duplicated()
        if duplicates.sum() > 0:
            print(f"Eliminando {duplicates.sum()} filas duplicadas...")
            data = data.drop_duplicates().reset_index(drop=True)
    
    # =============================================================================
    # 2.2 MANEJO DE VALORES FALTANTES
    # =============================================================================
    
    print("Manejando valores faltantes...")
    
    # Chequear valores faltantes antes del preprocesado
    missing_before = data.isnull().sum()
    print("Valores faltantes antes del preprocesado:")
    print(missing_before[missing_before > 0])
    
    # Imputar valores faltantes usando KNN para variables numéricas principales
    numeric_features = [
        'superficie_interior_m2', 
        'superficie_exterior_m2', 
        'distancia_centro_km', 
        'distancia_escola_km', 
        'indice_criminalidade'
    ]
    
    # Seleccionar columnas auxiliares para KNN (que tengan pocos o ningún NA)
    aux_features = [
        'numero_habitacions', 
        'numero_banos', 
        'ano_construccion',
        'lonxitude', 
        'latitude', 
        'temperatura_media_mes_construccion',
        'numero_arboles_xardin'
    ]
    
    # Combinar características para imputer
    imputer_features = numeric_features + aux_features
    
    # Crear una copia temporal para la imputación
    imputer_data = data[imputer_features].copy()
    
    # Inicializar KNN Imputer
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    
    # Aplicar imputación
    imputed_values = imputer.fit_transform(imputer_data)
    
    # Reemplazar valores en el DataFrame original solo para las columnas con NAs
    for i, col in enumerate(numeric_features):
        # Solo imputar si hay valores faltantes en la columna
        if data[col].isnull().sum() > 0:
            data[col] = imputed_values[:, i]
    
    # =============================================================================
    # 2.3 INGENIERÍA DE CARACTERÍSTICAS
    # =============================================================================
    
    print("Creando nuevas características...")
    
    # Año actual para cálculos de edad
    current_year = 2025
    
    # Crear características básicas
    data['edad_vivienda'] = current_year - data['ano_construccion']
    data['superficie_por_habitacion'] = data['superficie_interior_m2'] / data['numero_habitacions']
    data['superficie_total'] = data['superficie_interior_m2'] + data['superficie_exterior_m2']
    data['ratio_interior_exterior'] = data['superficie_interior_m2'] / (data['superficie_exterior_m2'] + 1)  # Evitar división por cero
    data['densidad_banos'] = data['numero_banos'] / data['superficie_interior_m2']
    data['densidad_habitaciones'] = data['numero_habitacions'] / data['superficie_interior_m2']
    
    # Características de localización
    # Calculamos la distancia euclidiana desde algunos puntos de referencia en Galicia
    # Centro aproximado de A Coruña
    data['dist_coruna'] = np.sqrt((data['lonxitude'] - (-8.4))**2 + (data['latitude'] - 43.37)**2)
    # Centro aproximado de Vigo
    data['dist_vigo'] = np.sqrt((data['lonxitude'] - (-8.72))**2 + (data['latitude'] - 42.23)**2)
    # Centro aproximado de Santiago
    data['dist_santiago'] = np.sqrt((data['lonxitude'] - (-8.54))**2 + (data['latitude'] - 42.88)**2)
    
    # Características interactivas
    data['calidad_edad'] = data['edad_vivienda'] * data.apply(
        lambda x: {'Alta': 3, 'Media': 2, 'Baixa': 1}[x['calidade_materiais']] 
        if pd.notna(x['calidade_materiais']) else 2, axis=1)
    
    data['banos_por_habitacion'] = data['numero_banos'] / data['numero_habitacions']
    
    # Características climáticas
    # Codificar la orientación según exposición solar (Sur es mejor)
    orientacion_map = {
        'Sur': 4,
        'Este': 3,
        'Oeste': 2,
        'Norte': 1
    }
    data['orientacion_valor'] = data['orientacion'].map(orientacion_map)
    
    # Codificar eficiencia energética
    eficiencia_map = {
        'A': 7,
        'B': 6,
        'C': 5,
        'D': 4,
        'E': 3,
        'F': 2,
        'G': 1
    }
    data['eficiencia_valor'] = data['eficiencia_enerxetica'].map(eficiencia_map)
    
    # Codificar calidad de materiales
    calidad_map = {
        'Alta': 3,
        'Media': 2,
        'Baixa': 1
    }
    data['calidade_valor'] = data['calidade_materiais'].map(calidad_map)
    
    # Codificar acceso a transporte público
    transporte_map = {
        'Bo': 3,
        'Regular': 2,
        'Malo': 1
    }
    data['transporte_valor'] = data['acceso_transporte_publico'].map(transporte_map)
    
    # Características de tipo de vivienda
    # One-hot encoding para tipo de edificación
    tipo_edificacion_dummies = pd.get_dummies(data['tipo_edificacion'], prefix='tipo')
    data = pd.concat([data, tipo_edificacion_dummies], axis=1)
    
    # One-hot encoding para color favorito del propietario
    color_dummies = pd.get_dummies(data['cor_favorita_propietario'], prefix='color')
    data = pd.concat([data, color_dummies], axis=1)
            
    # =============================================================================
    # 2.4 MANEJO DE OUTLIERS (Solo para entrenamiento)
    # =============================================================================
    
    if is_train:
        print("Analizando outliers...")
        
        # Detectar outliers en el precio
        Q1 = data['prezo_euros'].quantile(0.25)
        Q3 = data['prezo_euros'].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = data[(data['prezo_euros'] < lower_bound) | (data['prezo_euros'] > upper_bound)]
        
        print(f"Detectados {len(outliers)} outliers en el precio ({len(outliers)/len(data)*100:.2f}%)")
        
        # Decidimos mantener los outliers pero los marcamos para posible uso futuro
        data['is_outlier'] = (data['prezo_euros'] < lower_bound) | (data['prezo_euros'] > upper_bound)
        
        # Transformar la variable objetivo para normalización (log transform)
        data['log_prezo'] = np.log1p(data['prezo_euros'])
            
    # =============================================================================
    # 2.5 NORMALIZACIÓN Y ESCALADO
    # =============================================================================
    
    print("Escalando características numéricas...")
    
    # Lista de características numéricas para escalar
    numeric_features_to_scale = [
        'superficie_interior_m2', 
        'superficie_exterior_m2',
        'distancia_centro_km', 
        'distancia_escola_km',
        'indice_criminalidade',
        'superficie_por_habitacion',
        'superficie_total',
        'ratio_interior_exterior',
        'densidad_banos',
        'densidad_habitaciones',
        'dist_coruna',
        'dist_vigo',
        'dist_santiago',
        'calidad_edad'
    ]
    
    # No escalamos variables como número de habitaciones, baños, etc. que tienen sentido como están
    
    if is_train:
        # Inicializar el scaler
        scaler = StandardScaler()
        # Ajustar el scaler solo en los datos de entrenamiento
        scaler.fit(data[numeric_features_to_scale])
        # Guardar el scaler para uso futuro
        import joblib
        joblib.dump(scaler, 'scaler.pkl')
    else:
        # Cargar el scaler previamente ajustado
        import joblib
        try:
            scaler = joblib.load('scaler.pkl')
        except:
            print("ADVERTENCIA: No se encontró el scaler. Los datos de prueba no se escalarán correctamente.")
            return data
    
    # Aplicar la transformación
    data[numeric_features_to_scale] = scaler.transform(data[numeric_features_to_scale])
    
    # =============================================================================
    # 2.6 LIMPIAR COLUMNAS INNECESARIAS
    # =============================================================================
    
    print("Limpiando columnas innecesarias...")
    
    # Eliminar columnas originales que ya han sido procesadas o no son necesarias
    columns_to_drop = [
        # No eliminar 'id' ya que se necesita para la presentación
    ]
    
    data = data.drop(columns=columns_to_drop, errors='ignore')
    
    # Verificar valores faltantes después del preprocesado
    missing_after = data.isnull().sum()
    print("Valores faltantes después del preprocesado:")
    print(missing_after[missing_after > 0])
    
    return data

In [9]:
# =============================================================================
# 3. APLICAR PREPROCESAMIENTO
# =============================================================================

print("\nPreprocesando conjunto de entrenamiento...")
train_processed = preprocess_data(train_data, is_train=True)

print("\nPreprocesando conjunto de prueba...")
test_processed = preprocess_data(test_data, is_train=False)

# =============================================================================
# 4. GUARDAR DATOS PROCESADOS
# =============================================================================

print("\nGuardando datos procesados...")
train_processed.to_csv('train_processed.csv', index=False)
test_processed.to_csv('test_processed.csv', index=False)

print("Datos procesados guardados en 'train_processed.csv' y 'test_processed.csv'")



Preprocesando conjunto de entrenamiento...
Iniciando limpieza de datos...
Manejando valores faltantes...
Valores faltantes antes del preprocesado:
Series([], dtype: int64)
Creando nuevas características...
Analizando outliers...
Detectados 238 outliers en el precio (1.19%)
Escalando características numéricas...
Limpiando columnas innecesarias...
Valores faltantes después del preprocesado:
Series([], dtype: int64)

Preprocesando conjunto de prueba...
Iniciando limpieza de datos...
Manejando valores faltantes...
Valores faltantes antes del preprocesado:
Series([], dtype: int64)
Creando nuevas características...
Escalando características numéricas...
Limpiando columnas innecesarias...
Valores faltantes después del preprocesado:
Series([], dtype: int64)

Guardando datos procesados...
Datos procesados guardados en 'train_processed.csv' y 'test_processed.csv'


In [10]:
# =============================================================================
# 5. VISUALIZACIÓN DE RESULTADOS DEL PREPROCESAMIENTO
# =============================================================================

print("\nMostrando información de los datos procesados:")
print(f"Dimensiones del conjunto de entrenamiento procesado: {train_data.shape}")
print(f"Dimensiones del conjunto de prueba procesado: {test_data.shape}")

# Mostrar las primeras filas
print("\nPrimeras filas del conjunto de entrenamiento procesado:")
print(train_data.head())

# Mostrar las columnas disponibles
print("\nColumnas disponibles en el conjunto de entrenamiento procesado:")
print(train_data.columns.tolist())

# Verificar nuevas características creadas
new_features = ['superficie_por_habitacion', 'edad_vivienda', 'superficie_total', 
                'ratio_interior_exterior', 'dist_coruna', 'dist_vigo', 'dist_santiago']
print("\nEstadísticas de las nuevas características:")
print(train_data[new_features].describe())

print("\nPreprocesamiento completado exitosamente.")


Mostrando información de los datos procesados:
Dimensiones del conjunto de entrenamiento procesado: (20000, 46)
Dimensiones del conjunto de prueba procesado: (10000, 44)

Primeras filas del conjunto de entrenamiento procesado:
      id  superficie_interior_m2  superficie_exterior_m2  numero_habitacions  \
0  25521               -1.214878               -0.600905                   1   
1   4843               -0.590517               -0.493874                   2   
2  27734                1.723563               -0.371180                   1   
3  22142                0.415219               -0.533578                   4   
4  14748                1.203970                2.040511                   1   

   numero_banos  ano_construccion  lonxitude  latitude  \
0             2              1947      -8.17     43.20   
1             2              1977      -7.23     43.60   
2             1              1996      -8.40     42.25   
3             2              1996      -6.81     43.15   
4